In [1]:
-- import the necessary torch libraries
require 'rnn'
require 'optim'

In [2]:
-- first define a sequence just for demo
data = string.lower("Singapore, officially the Republic of Singapore, sometimes referred to as the Lion City or the Little Red Dot, is a sovereign city state in Southeast Asia. It lies one degree (137 km) north of the equator, at the southern tip of peninsular Malaysia, with Indonesia's Riau Islands to the south. Singapore's territory consists of one main island along with 62 other islets. Since independence, extensive land reclamation has increased its total size by 23% (130 km2), and its greening policy has covered the densely populated island with tropical flora, parks and gardens. Stamford Raffles founded colonial Singapore in 1819 as a trading post of the East India Company; after its collapse and the eventual establishment of the British Raj, the islands were ceded to Britain and became part of its Straits Settlements in 1826. During the Second World War, Singapore was occupied by Japan. It gained independence from the UK in 1963 by federating with other former British territories to form Malaysia, but was expelled two years later over ideological differences, becoming a sovereign nation in 1965. After early years of turbulence, and despite lacking natural resources and a hinterland, the nation developed rapidly as an Asian Tiger economy, based on external trade and its workforce. Singapore is a global commerce, finance and transport hub. Its standings include: the most technology ready nation (WEF), top International-meetings city (UIA), city with best investment potential (BERI), second-most competitive country, third-largest foreign exchange market, third-largest financial centre, third-largest oil refining and trading centre, and the second-busiest container port. The country has also been identified as a tax haven. Singapore ranks 5th internationally and first in Asia on the UN Human Development Index, and 3rd highest per capita income. It is ranked highly in education, healthcare, life expectancy, quality of life, personal safety, and housing. Although income inequality is high, 90% of homes are owner-occupied. 38% of Singapore's 5.6 million residents are permanent residents and other foreign nationals. There are four official languages: English (common and first language), Malay, Mandarin, Tamil; almost all Singaporeans are bilingual. Singapore is a unitary multiparty parliamentary republic, with a Westminster system of unicameral parliamentary government. The People's Action Party has won every election since self-government in 1959. The dominance of the PAP, coupled with a low level of press freedom and restrictions on civil liberties and political rights, has led to Singapore being classified by some as a semi-authoritarian regime. One of the five founding members of the ASEAN, Singapore is also the host of the Asia-Pacific Economic Cooperation (APEC) Secretariat, and a member of the East Asia Summit, Non-Aligned Movement, and the Commonwealth of Nations.")

In [5]:
#data

2902	


In [ ]:
data = string.lower("Singapore, officially the Republic of Singapore, sometimes referred to as the Lion City or the Little Red Dot, is a sovereign city state in Southeast Asia.")

In [3]:
-- now we have to read in the characters and assign an index to each character
char_index = {} -- a hash table
input = {}
target = {}
i = 0
j = 0
for c in data:gmatch"." do -- this line is to iterate the characters in data
  if char_index[c] == nil then -- then is always necessary in a if statement
    i = i+1
    char_index[c] = i
  end
  j = j+1
  input[j] = torch.LongTensor({char_index[c]})
  if j >= 2 then
    target[j-1] = input[j]
  end
end

-- add the end-of-sequence token
i = i+1
char_index['<EOS>'] = i

-- add the EOS token at end of output
target[j] = torch.LongTensor({char_index['<EOS>']})
vocab_size = i -- total vocab size

In [4]:
target = nn.Sequential()
  :add(nn.JoinTable(1)) -- convert table into tensor
  :forward(target)

In [5]:
vocab_size

46	


In [6]:
K=92 -- size of the embedding dimensions
lk = nn.LookupTable(vocab_size, K)
num_layers = 2

In [ ]:
-- now create a neural network architecture that will read from input and update the weights in the lstm cells
char_lstm = nn.Sequential() -- create the container for this architecture
  :add(nn.Sequencer(lk)) -- add the embedding layer to this
  :add(nn.JoinTable(1)) -- convert table into tensor
  :add(nn.Unsqueeze(2))

char_lstm.num_layers = num_layers
char_lstm.layers = {}
for i=1,num_layers do
  char_lstm.layers[i] = nn.SeqLSTM(K, K)
  char_lstm:add(char_lstm.layers[i])
  char_lstm:add(nn.Sequencer(nn.Tanh()))
end
  
upper_layers = nn.Sequential()
  :add(nn.Linear(K, vocab_size))
  :add(nn.LogSoftMax())

char_lstm:add(nn.Sequencer(upper_layers))

In [7]:
criterion = nn.SequencerCriterion(nn.ClassNLLCriterion()) -- the loss function, negative log likelihood

In [9]:
char_lstm = torch.load('char_lstm.model.ascii', 'ascii')

In [10]:
params, gradParams = char_lstm:getParameters()

In [11]:
config = {
  learningRate = 1e-3,
--   weightDecay = 1e-3, --l2 regularization
  momentum = 0.9
}
timer = torch.Timer()

In [2]:
total_iteration = 200
for iteration = 1,total_iteration do  
  function feval(params)
    gradParams:zero()
    local pred = char_lstm:forward(input)
    local err = criterion:forward(pred, target)
    
    local grad_output = criterion:backward(pred, target)
    local grad_input = char_lstm:backward(input, grad_output)
    return err, gradParams
  end
  params, err = optim.sgd(feval, params, config)
  if iteration % 10 == 0 then
    local perplexity = torch.exp(err[1] / target:size(1))
    print(string.format("Iteration %d: NLL = %f, perplexity = %f, elapsed time %f secs", 
      iteration, err[1], perplexity, timer:time().real))
    timer:reset()
  end
end
timer:stop()

Iteration 10: NLL = 76.857857, perplexity = 1.026838, elapsed time 0.000000 secs	


Iteration 20: NLL = 74.194774, perplexity = 1.025896, elapsed time 0.000000 secs	


Iteration 30: NLL = 71.684321, perplexity = 1.025009, elapsed time 0.000000 secs	


Iteration 40: NLL = 69.315313, perplexity = 1.024173, elapsed time 0.000000 secs	


Iteration 50: NLL = 67.077630, perplexity = 1.023383, elapsed time 0.000000 secs	


...

Iteration 160: NLL = 48.918073, perplexity = 1.017000, elapsed time 0.000000 secs	


Iteration 170: NLL = 47.701073, perplexity = 1.016573, elapsed time 0.000000 secs	


Iteration 180: NLL = 46.537303, perplexity = 1.016166, elapsed time 0.000000 secs	


Iteration 190: NLL = 45.423507, perplexity = 1.015776, elapsed time 0.000000 secs	


Iteration 200: NLL = 44.356666, perplexity = 1.015402, elapsed time 0.000000 secs	


In [13]:
index_char = {}
for k,v in pairs(char_index) do
  index_char[v] = k
end

In [1]:
torch.save('char_lstm.model.ascii', char_lstm, 'ascii')

In [14]:
-- beam search
function beam_search(model, beam_size, max_len, start_symbol, end_symbol)
  model:forget()
  model:remember('neither')
  for i=1,num_layers do
    model.layers[i].userPrevCell   = nil
    model.layers[i].userPrevOutput = nil
  end
  
  local current = start_symbol
  local candidates = {} -- maintain top sequences according to beam_size
  
  table.insert(candidates, {
      ['str'] = {[1] = current},
      ['logll'] = 0.0,
      ['len'] = 1
    })
  
  for i=2,max_len do
    -- go through the candidates, for each candidate, expand by beam_size, add it to list of candidates
    local new_candidates = {}
    for j=1,#candidates do
      -- transfer the previous states of lstm to the current lstm
      if i>2 then
        for k=1,num_layers do
          model.layers[k].userPrevCell   = candidates[j].userPrevCell[k]
          model.layers[k].userPrevOutput = candidates[j].userPrevOutput[k]
        end
      end
      -- get the last predicted entry
      local current = candidates[j].str[candidates[j].len]
      if current ~= end_symbol then
        local log_prob = model:forward({torch.LongTensor({current})})
        local sorted_log_prob, idx = torch.topk(log_prob[1][1], beam_size, 1, true)
        for n=1,beam_size do
          local current_candidate = {
            ['str'] = { table.unpack(candidates[j].str) },
            ['logll'] = candidates[j].logll + sorted_log_prob[n],
            ['userPrevOutput'] = {},
            ['userPrevCell'] = {}
          }
          table.insert(current_candidate.str, idx[n])
          current_candidate.len = #current_candidate.str
          current_candidate.perplexity = torch.exp(-current_candidate.logll / current_candidate.len)
          for k=1, num_layers do
            current_candidate.userPrevCell[k] = model.layers[k].cell[1]:clone()
            current_candidate.userPrevOutput[k] = model.layers[k].output[1]:clone()
          end
          table.insert(new_candidates, current_candidate)
        end
      end
    end
--     then sort candidates and only retain the top beam_sizes
    table.sort(new_candidates, function (c1, c2) return c1.perplexity < c2.perplexity end)
    -- add topk to candidates
    candidates = {}
    for i=1,math.min(beam_size, #new_candidates) do
      table.insert(candidates, new_candidates[i])
    end
  end
--   print(candidates)
  return candidates
end

In [6]:
results = beam_search(char_lstm, 3, 3000, 1, char_index['<EOS>'])

In [7]:
for i=1,#results do
  str = ''
  for _, idx in ipairs(results[i].str) do
    str = str .. index_char[idx]
  end
  print(i, str)
end

1	singapore, officially the republic of singapore, sometimes referred to as the lion city or the little red dot, is a sovereign city state in southeast asia. it lies one degree (137 km) north of the equator, at the southern tip of peninsular malaysia, with indonesia's riau islands to the south. singapore's territory consists of one main island along with 62 other islets. since independence, extensive land reclamation has increased its total size by 23% (130 km2), and its greening policy has covered the densely populated island with tropical flora, parks and gardens. stamford raffles founded colonial singapore in 1819 as a trading post of the east india company; after its collapse and the eventual establishment of the british raj, the islands were ceded to britain and became part of its straits settlements in 1826. during the second world war, singapore was occupied by japan. it gained independence from the uk in 1963 by federating with other former british territories to form malaysia,

2	singapore, officially the republic of singapore, sometimes referred to as the lion city or the little red dot, is a sovereign city state in southeast asia. it lies one degree (137 km) north of the equator, at the southern tip of peninsular malaysia, with indonesia's riau islands to the south. singapore's territory consists of one main island along with 62 other islets. since independence, extensive land reclamation has increased its total size by 23% (130 km2), and its greening policy has covered the densely populated island with tropical flora, parks and gardens. stamford raffles founded colonial singapore in 1819 as a trading post of the east india company; after its collapse and the eventual establishment of the british raj, the islands were ceded to britain and became part of its straits settlements in 1826. during the second world war, singapore was occupied by japan. it gained independence from the uk in 1963 by federating with other former british territories to form malaysia,

3	singapore, officially the republic of singapore, sometimes referred to as the lion city or the little red dot, is a sovereign city state in southeast asia. it lies one degree (137 km) north of the equator, at the southern tip of peninsular malaysia, with indonesia's riau islands to the south. singapore's territory consists of one main island along with 62 other islets. since independence, extensive land reclamation has increased its total size by 23% (130 km2), and its greening policy has covered the densely populated island with tropical flora, parks and gardens. stamford raffles founded colonial singapore in 1819 as a trading post of the east india company; after its collapse and the eventual establishment of the british raj, the islands were ceded to britain and became part of its straits settlements in 1826. during the second world war, singapore was occupied by japan. it gained independence from the uk in 1963 by federating with other former british territories to form malaysia,

In [17]:
function sample(log_prob)
  rand_p = torch.uniform()
  cum_p = torch.FloatTensor(log_prob:size())
  cum_p[1] = torch.exp(log_prob[1])
  for i=1,log_prob:size(1)-1 do
    if rand_p <= cum_p[i] then
      return i
    else
      cum_p[i+1] = cum_p[i] + torch.exp(log_prob[i+1])
    end
  end
  return log_prob:size(1)
end

In [30]:
-- generating by sampling
local c = 's'
local str = c
local c_idx = char_index[c]
for i=1,100 do
  local log_prob = char_lstm:forward({torch.LongTensor({c_idx})})
  for k=1,num_layers do
    char_lstm.layers[k].userPrevCell   = char_lstm.layers[k].cell[-1]
    char_lstm.layers[k].userPrevOutput = char_lstm.layers[k].output[-1]
  end
  log_prob = torch.squeeze(log_prob)
  c_idx = sample(log_prob)
  str = str .. index_char[c_idx]
end
print(str)

singapore, (cmetto fanthorial singapore former wa) citer evensive of the fom the un idenderents and o	
